Ön hazırlık (kütüphane kurulumu)

In [2]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.0 MB/s eta 0:00:00


Soru 1 – Bell Durumu Oluşturma ($\ket{00} \xrightarrow{H + CNOT} \ket{B_{00}}$)
Cirq

In [3]:
import cirq
import numpy as np

# Qubitleri tanımla
q0, q1 = cirq.LineQubit.range(2)

# Devre oluştur
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1)
)

# Devreyi göster
print("Cirq Devresi:")
print(circuit)

# Simülasyon
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
print("\nSon Durum:")
print(result.final_state_vector)


Cirq Devresi:
0: ───H───@───
          │
1: ───────X───

Son Durum:
[0.70710677+0.j 0.        +0.j 0.        +0.j 0.70710677+0.j]


Qiskit

In [4]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.quantum_info import Statevector

# Devre oluştur
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Devreyi göster
print("Qiskit Devresi:")
print(qc)

# Durumu simüle et
state = Statevector.from_instruction(qc)
print("\nSon Durum:")
print(state.data)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP

In [5]:
from qutip import tensor, basis, hadamard_transform, cnot

# Başlangıç durumu |00>
psi0 = tensor(basis(2,0), basis(2,0))

# Hadamard ilk qubit
H = hadamard_transform()
psi1 = tensor(H*psi0.ptrace(0), psi0.ptrace(1))

# CNOT
psi_final = cnot() * psi1

print("QuTiP Son Durum:")
print(psi_final)


ImportError: cannot import name 'hadamard_transform' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

Soru 2 – Bell Durumunun Tersine Mühendisliği ($\ket{B_{10}}$)

Başlangıç durumu: $\ket{10}$, ardından H ve CNOT.

Cirq

In [6]:
q0, q1 = cirq.LineQubit.range(2)

circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1)
)

simulator = cirq.Simulator()
psi0 = np.array([0,0,1,0])  # |10>
result = simulator.simulate(circuit, initial_state=psi0)
print("Cirq - B10 Son Durum:")
print(result.final_state_vector)


Cirq - B10 Son Durum:
[ 0.70710677+0.j  0.        +0.j -0.        +0.j -0.70710677+0.j]


Qiskit

In [7]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Başlangıç |10>
init_state = [0,0,1,0]
state = Statevector(init_state).evolve(qc)
print("Qiskit - B10 Son Durum:")
print(state.data)


Qiskit - B10 Son Durum:
[0.        +0.j 0.70710678+0.j 0.70710678+0.j 0.        +0.j]


QuTiP

In [8]:
from qutip import basis, tensor, hadamard_transform, cnot

psi0 = tensor(basis(2,1), basis(2,0))  # |10>
H = hadamard_transform()
psi1 = tensor(H*psi0.ptrace(0), psi0.ptrace(1))
psi_final = cnot() * psi1

print("QuTiP - B10 Son Durum:")
print(psi_final)


ImportError: cannot import name 'hadamard_transform' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

Soru 3 – Bell Durumu Ayrışımı (Dolanıklık Testi)

Durum: $\ket{\phi} = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$

Cirq

In [9]:
import cirq
import numpy as np

# Durum vektörü
phi = np.array([0, 1/np.sqrt(2), 1/np.sqrt(2), 0])
print("Cirq - Phi Durumu:")
print(phi)

# Dolaşıklık kontrolü (sadece Schmidt katsayıları)
u, s, vh = np.linalg.svd(phi.reshape(2,2))
print("Schmidt katsayıları:", s)


Cirq - Phi Durumu:
[0.         0.70710678 0.70710678 0.        ]
Schmidt katsayıları: [0.70710678 0.70710678]


Qiskit

In [10]:
import numpy as np
from qiskit.quantum_info import DensityMatrix, partial_trace

phi = np.array([0, 1/np.sqrt(2), 1/np.sqrt(2), 0])
rho = DensityMatrix(phi)
rho_A = partial_trace(rho, [1])  # 1. qubit'i trace et

print("Qiskit - Rho_A eigenvalues (dolanıklık kontrolü):")
print(np.linalg.eigvals(rho_A.data))


Qiskit - Rho_A eigenvalues (dolanıklık kontrolü):
[0.5+0.j 0.5+0.j]


QuTiP

In [11]:
from qutip import basis, tensor, Qobj, ptrace

phi = (tensor(basis(2,0), basis(2,1)) + tensor(basis(2,1), basis(2,0))).unit()
rho_A = ptrace(phi, 0)  # 1. qubit'i trace et
print("QuTiP - Phi Durumu:")
print(phi)
print("rho_A:")
print(rho_A)


QuTiP - Phi Durumu:
Quantum object: dims=[[2, 2], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.        ]
 [0.70710678]
 [0.70710678]
 [0.        ]]
rho_A:
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5 0. ]
 [0.  0.5]]
